# TerrainPatterns
Let's patternd the world

In [ ]:
#| default_exp terrainpatterns

### To Do's

1. Create a new resource group for triangles
1. Fix contour so it only does the high side

### Prior Art

In [ ]:
#| export
#standard
import numpy as np
import sys
import os
import math
import random

#data
from collections import namedtuple
from dataclasses import dataclass,  field, asdict
from typing import List
from enum import Enum

#Jeremy
from dialoghelper import * 
from fastcore.basics import patch
from fasthtml.common import *
from fasthtml.jupyter import *
import httpx

# unique
from treelib import Tree
import heapq
from HexMagic.voronoi import generate_plate_terrain



In [ ]:
#| export

from HexMagic.styles import StyleCSS, SVGBuilder,SVGLayer, SVGPatternLoader, preview, app, StyleDemo, Generatable,SVGDef
from HexMagic.primitives import MapCord, MapSize, MapRect, MapPath, Hex, HexGrid, HexRegion, HexWrapper, HexPosition, PrimitiveDemo, hexBackground, windy_edge, unique_windy_edge

In [ ]:
#| export
from HexMagic.terrain import Terrain, TerraDemo
Terrain.fromSeeds = generate_plate_terrain

### Plot Thickens

Based on the code context, here's a summary of how overlay plotting works in this terrain visualization system:

**Overlay Plotting Summary**

The overlay system allows you to visualize additional data layers (like rainfall, temperature) on top of the base terrain elevation map using SVG patterns.

**Key Components:**

1. **Pattern Generation** (`ballPatterns`): Creates a set of SVG pattern definitions, typically representing different intensity levels. Each pattern is a scaled circle whose radius increases with the level index.

2. **Data Binning** (`rainfall_selector_np`): Converts continuous data values into discrete pattern indices using `np.digitize`. For example, rainfall amounts are binned into categories (0-5, 5-12, 12-24, 24-48, 48+ inches).

3. **Overlay Application** (`makeOverlay`): 
   - Adds pattern definitions to the SVG builder
   - Iterates through each hex cell
   - Applies the appropriate pattern as a fill based on the data index
   - Creates polygon elements with `url(#pattern_id)` fills

**Workflow:**
```python
# 1. Prepare your data field
rainfall_data = np.random.uniform(-4, 60, len(terrain.elevations))
terrain.fields['rainfall'] = rainfall_data

# 2. Bin the data into pattern indices
pattern_indices = rainfall_selector_np(rainfall_data)

# 3. Generate patterns and apply overlay
patterns = ballPatterns(5)  # 5 intensity levels
terrain.makeOverlay(pattern_indices, patterns)
```

The result is a dual-layer visualization: colored hexes showing elevation beneath patterned overlays showing your additional data field.

In [ ]:
#| export
class TerrainPatterns:

    def __init__(self,terrain):
        self.terrain = terrain

    @classmethod
    def circlePattern(cls, id, radius=5, spacing=10, color="black"):
        """Generate a circle pattern definition"""
        content = f'<circle cx="{spacing/2}" cy="{spacing/2}" r="{radius}" fill="{color}"/>'
        return SVGDef("pattern", id, content, 
                    width=spacing, height=spacing, 
                    patternUnits="userSpaceOnUse")

    @classmethod
    def trianglePattern(cls, id, size=10, spacing=12, color="black"):
        """Generate an equilateral triangle pattern"""
        height = size * 0.866  # sqrt(3)/2
        points = f"0,{height} {size/2},0 {size},{height}"
        content = f'<polygon points="{points}" fill="{color}"/>'
        return SVGDef("pattern", id, content,
                    width=spacing, height=spacing,
                    patternUnits="userSpaceOnUse")

    @classmethod
    def wavePattern(cls, id, amplitude=5, wavelength=20, color="black", stroke_width=2):
        """Generate a wave pattern"""
        path = f"M 0,{amplitude} Q {wavelength/4},0 {wavelength/2},{amplitude} T {wavelength},{amplitude}"
        content = f'<path d="{path}" stroke="{color}" stroke-width="{stroke_width}" fill="none"/>'
        return SVGDef("pattern", id, content,
                    width=wavelength, height=amplitude*2,
                    patternUnits="userSpaceOnUse")

    @classmethod
    def crosshatchPattern(cls, id, spacing=10, color="black", stroke_width=1):
        """Generate a crosshatch pattern"""
        content = f'<line x1="0" y1="0" x2="{spacing}" y2="{spacing}" stroke="{color}" stroke-width="{stroke_width}"/>'
        content += f'<line x1="{spacing}" y1="0" x2="0" y2="{spacing}" stroke="{color}" stroke-width="{stroke_width}"/>'
        return SVGDef("pattern", id, content,
                    width=spacing, height=spacing,
                    patternUnits="userSpaceOnUse")



In [ ]:
#| export
@patch
def ballDensity(self:TerrainPatterns, levels=6, fills=["#007fff"], prefix="ball") -> list:
    """Create density patterns using circles of increasing size."""
    ret = []
    spacing = 6  # scaled down from 60 with 0.1 scale
    
    for i in range(levels):
        colorIndex = min(i, len(fills)-1)
        fill = fills[colorIndex]
        radius = min(i * 5.6 / levels + 0.2, spacing/2 - 0.1)
        
        pattern = self.circlePattern(f"{prefix}_{i}", radius=radius, spacing=spacing, color=fill)
        ret.append(pattern)
    return ret

@patch
def ballSpectrum(self:TerrainPatterns, levels=5, fills=["#007fff"], prefix="ball") -> list:
    """Create density patterns - circles shrink to middle then grow."""
    ret = []
    spacing = 6
    biggest = spacing/2 - 0.1
    smallest = 0.5
    delta = (biggest - smallest) / levels * 2
    
    for i in range(levels):
        colorIndex = min(i, len(fills)-1)
        fill = fills[colorIndex]
        
        if i < levels/2:
            radius = biggest - delta * i
        elif i > levels/2:
            k = levels - i
            radius = biggest - delta * k
        else:
            radius = smallest
        
        pattern = self.circlePattern(f"{prefix}_{i}", radius=radius, spacing=spacing, color=fill)
        ret.append(pattern)
    return ret


reference code
```
class SVGDef(Generatable):
    """ This is the concrete class that is the simplest implementation of Generatable. It just stores the content in a string. More complex version of Generatable will compute this on the fly ."""

    def __init__(self, tag, id, content,  **attributes): 
        self.content = content
        self.tag_name = tag  # Renamed to avoid shadowing the class method
        if attributes is None:
            attributes = {}
        attributes["id"] = id
        self.attributes = attributes

    def generate(self):
        return tag(self.tag_name, self.content, **self.attributes)


    def display(self):
        ret = f"+{self.tag_name} Header\n"

        for key in sorted(self.attributes.keys()):
            value = self.attributes[key]
            
            ret += f"\t{key}:{value}"
            ret += "\n"
        ret += f"-{self.tag_name} Header\n\n"
        ret += f"+{self.tag_name} Body\n"
        ret += self.content
        ret += f"\n-{self.tag_name} Body\n"

        return ret

    def __str__(self):
        return self.generate()

    def __repr__(self):
        return self.display()
```

I was thinking about extending TerrainPattern so that it can give us some geometric shapes as patterns for svg files. we do have SVGDef which we could use by doing its body and tag properties. so the idea would be things that are good for fills like
1. circles.
2. triangles.
3. waves
4. cross hatches.

we might want to eventually add other textures to represent climates like :
Climate.MARINE.value: "#1e88e5",
Climate.FRESHWATER.value: "#42a5f5",
Climate.TUNDRA.value: "#e3f2fd",
Climate.DESERT.value: "#fdd835",
Climate.GRASSLAND.value: "#9ccc65",
Climate.FOREST.value: "#2e7d32",
Climate.JUNGLE.value: "#1b5e20",
or raindrops for showing weather


##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

This is a nice extension idea! Let's start by creating a method that generates SVG pattern definitions for geometric shapes.

Looking at your `SVGDef` reference code, we need to create pattern elements with a specific structure. For SVG patterns, the basic approach is:

```python
def circlePattern(self, id, radius=5, spacing=10, color="black"):
    """Generate a circle pattern definition"""
    content = Circle(cx=spacing/2, cy=spacing/2, r=radius, fill=color)
    return SVGDef("pattern", id, content, 
                  width=spacing, height=spacing, 
                  patternUnits="userSpaceOnUse")
```

This would create a repeating circle pattern. The key attributes are:
- `width` and `height` define the pattern tile size
- `patternUnits="userSpaceOnUse"` makes it work in your coordinate system
- The content is the shape(s) inside each tile

Before I write more methods, a couple of questions:

1. Should `TerrainPatterns` store a collection of pattern definitions, or generate them on demand?
2. Do you want each pattern method to return an `SVGDef`, or should there be a method that adds them to a collection?

So we don't svg class so we need things like
  <circle cx="50" cy="50" r="50" class="circleCs"/>


##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Ah right, you need to generate the raw SVG string! So instead of using a `Circle` class, you'd build the string directly:

```python
def circlePattern(self, id, radius=5, spacing=10, color="black"):
    """Generate a circle pattern definition"""
    content = f'<circle cx="{spacing/2}" cy="{spacing/2}" r="{radius}" fill="{color}"/>'
    return SVGDef("pattern", id, content, 
                  width=spacing, height=spacing, 
                  patternUnits="userSpaceOnUse")
```

Does this match what you need? Or would you prefer a different approach for building the SVG strings?

This is perfect. can we do some of the other primitives and then maybe think about climate.

    circles.
    triangles.
    waves
    cross hatches.

we might want to eventually add other textures to represent climates like : Climate.MARINE.value: "#1e88e5", Climate.FRESHWATER.value: "#42a5f5", Climate.TUNDRA.value: "#e3f2fd", Climate.DESERT.value: "#fdd835", Climate.GRASSLAND.value: "#9ccc65", Climate.FOREST.value: "#2e7d32", Climate.JUNGLE.value: "#1b5e20", or raindrops for showing weather

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! Let's add the other geometric primitives. Here are the methods to add to your `TerrainPatterns` class:

```python
def trianglePattern(self, id, size=10, spacing=12, color="black"):
    """Generate an equilateral triangle pattern"""
    height = size * 0.866  # sqrt(3)/2
    points = f"0,{height} {size/2},0 {size},{height}"
    content = f'<polygon points="{points}" fill="{color}"/>'
    return SVGDef("pattern", id, content,
                  width=spacing, height=spacing,
                  patternUnits="userSpaceOnUse")

def wavePattern(self, id, amplitude=5, wavelength=20, color="black", stroke_width=2):
    """Generate a wave pattern"""
    path = f"M 0,{amplitude} Q {wavelength/4},0 {wavelength/2},{amplitude} T {wavelength},{amplitude}"
    content = f'<path d="{path}" stroke="{color}" stroke-width="{stroke_width}" fill="none"/>'
    return SVGDef("pattern", id, content,
                  width=wavelength, height=amplitude*2,
                  patternUnits="userSpaceOnUse")

def crosshatchPattern(self, id, spacing=10, color="black", stroke_width=1):
    """Generate a crosshatch pattern"""
    content = f'<line x1="0" y1="0" x2="{spacing}" y2="{spacing}" stroke="{color}" stroke-width="{stroke_width}"/>'
    content += f'<line x1="{spacing}" y1="0" x2="0" y2="{spacing}" stroke="{color}" stroke-width="{stroke_width}"/>'
    return SVGDef("pattern", id, content,
                  width=spacing, height=spacing,
                  patternUnits="userSpaceOnUse")
```

Try adding these to your class. Once you have them working, we can think about how to map these patterns to climate types. Would you like to test these first, or should we move on to the climate mapping?

@patch
def ballScale(self, levels=6,fills=["#007fff"],prefix="ball")->[SVGBuilder]:
    ret = []
    for i in range(levels):
        colorIndex = min(i,len(fills)-1)
        fill = fills[colorIndex]
        
    
        ballDim = 60
        style = StyleCSS(f"{prefix}_{i}",fill=fill)
        aBuilder = SVGBuilder()
        radius = 20
        
        body = f"""
        <g>
            <ellipse cx="{ballDim/2}" cy="{ballDim/2}" rx="{radius}" ry="{radius}" style="fill:{fill};"/>
        </g>
    """
        aBuilder.blockTag = "pattern"
        aBuilder.width = ballDim
        aBuilder.height = ballDim
        scale = 0.1 + i * 0.8 / levels
        aBuilder.attributes = {
            'id': f"fill_{i}",
            'patternUnits': 'userSpaceOnUse',
            'patternTransform': f'scale({scale})'
        }
        aBuilder.updateLayers([body])

        ret.append(aBuilder)
    return ret

#| export
@patch
def ballDensity(self:TerrainPatterns, levels=6, fills=["#007fff"], prefix="ball") -> [SVGBuilder]:
    """Create density patterns using circles of increasing size."""
    ret = []
    for i in range(levels):
        colorIndex = min(i, len(fills)-1)
        fill = fills[colorIndex]
    
        ballDim = 60
        aBuilder = SVGBuilder()
        radius = min(i*56/levels + 2, ballDim/2-1)
        
        body = f"""
        <g>
            <circle cx="{ballDim/2}" cy="{ballDim/2}" r="{radius}" style="fill:{fill};"/>
        </g>
        """
        aBuilder.blockTag = "pattern"
        aBuilder.width = ballDim
        aBuilder.height = ballDim
        scale = 0.1
        aBuilder.attributes = {
            'id': f"{prefix}_{i}",  # USE THE PREFIX!
            'patternUnits': 'userSpaceOnUse',
            'patternTransform': f'scale({scale})'
        }
        aBuilder.updateLayers([body])

        ret.append(aBuilder)
    return ret

In [ ]:
#| export
@patch
def ballSpectrum(self:TerrainPatterns, levels=5, fills=["#007fff"], prefix="ball") -> [SVGBuilder]:
    """Create density patterns using circles of increasing size."""
    ret = []
    

    for i in range(levels):
        colorIndex = min(i, len(fills)-1)
        fill = fills[colorIndex]
    
        ballDim = 60
        biggest = ballDim/2-1 
        smallest = 5
        delta = (biggest - smallest)/levels * 2
        

        aBuilder = SVGBuilder()
        if i < levels/2:
            radius = biggest - delta * i
        elif i > levels/2:
            k = levels - i
            radius = biggest - delta * k
        else:
            radius = smallest



        
        body = f"""
        <g>
            <circle cx="{ballDim/2}" cy="{ballDim/2}" r="{radius}" style="fill:{fill};"/>
        </g>
        """
        aBuilder.blockTag = "pattern"
        aBuilder.width = ballDim
        aBuilder.height = ballDim
        scale = 0.1
        aBuilder.attributes = {
            'id': f"{prefix}_{i}",  # USE THE PREFIX!
            'patternUnits': 'userSpaceOnUse',
            'patternTransform': f'scale({scale})'
        }
        aBuilder.updateLayers([body])

        ret.append(aBuilder)
    return ret

I was thinking about rewriting ballSpectrum, ballDensity so that they return SVGDef and take advantage of these methods

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Great idea! Here's how you could rewrite them using `circlePattern`:

```python
@patch
def ballDensity(self:TerrainPatterns, levels=6, fills=["#007fff"], prefix="ball") -> list:
    """Create density patterns using circles of increasing size."""
    ret = []
    spacing = 6  # scaled down from 60 with 0.1 scale
    
    for i in range(levels):
        colorIndex = min(i, len(fills)-1)
        fill = fills[colorIndex]
        radius = min(i * 5.6 / levels + 0.2, spacing/2 - 0.1)
        
        pattern = self.circlePattern(f"{prefix}_{i}", radius=radius, spacing=spacing, color=fill)
        ret.append(pattern)
    return ret

@patch
def ballSpectrum(self:TerrainPatterns, levels=5, fills=["#007fff"], prefix="ball") -> list:
    """Create density patterns - circles shrink to middle then grow."""
    ret = []
    spacing = 6
    biggest = spacing/2 - 0.1
    smallest = 0.5
    delta = (biggest - smallest) / levels * 2
    
    for i in range(levels):
        colorIndex = min(i, len(fills)-1)
        fill = fills[colorIndex]
        
        if i < levels/2:
            radius = biggest - delta * i
        elif i > levels/2:
            k = levels - i
            radius = biggest - delta * k
        else:
            radius = smallest
        
        pattern = self.circlePattern(f"{prefix}_{i}", radius=radius, spacing=spacing, color=fill)
        ret.append(pattern)
    return ret
```

Note: I adjusted the dimensions since the old code used `scale(0.1)` on a 60-unit pattern. Now they work at the native scale (spacing=6). If you need the larger pattern with transform, you could add a `patternTransform` attribute to the `SVGDef` after creation.

In [ ]:
@patch
def dotDemo(self:TerraDemo):
    sampleMap = TerraDemo().tiny()
    terrainPatterns = TerrainPatterns(sampleMap)
    for x in terrainPatterns.ballDensity():
        print(x)

    

In [ ]:
TerraDemo().dotDemo()

In [ ]:
#| export
@patch
def makeOverlay(self:Terrain,data,patterns:[SVGBuilder])->str:
    testBody = ""
    grid = self.hexGrid

    addSet = set()

    for i, patIndex in enumerate(data):
        if patIndex >= 0 and patIndex < len(patterns):
            patName = patterns[patIndex].attributes['id']
            addSet.add(patIndex)
    
            fill = f"url(#{patName})"

            ret = "<polygon points=\""
            hex = grid.hexes[i]
            for point in hex.vertices():
                ret += f"{point.x:.0f},{point.y:.0f} "
            ret += f"\" style=\"fill:{fill}\""
            ret += "/>"

            testBody += "\t" +  ret + "\n"

    aList = list(addSet)
    aList.sort()
    for patIndex in aList:
        grid.builder.add_definition(patterns[patIndex])

    return testBody
   
        

In [ ]:
#| export
@patch
def overlayRegions(self: HexGrid, regions: list[HexRegion], 
                   patterns: list[SVGBuilder], f=None):
    """Apply patterns to regions. Each region gets one pattern applied to all its paths.
    
    Args:
        regions: List of HexRegion objects
        patterns: List of SVGBuilder patterns (one per region)
        f: Optional edge transform function (e.g., windy_edge())
    
    Returns:
        SVG string for the overlay layer
    """
    retLayer = ""
    borders = {}  # Shared cache across all regions
    
    # Add all pattern definitions
    for pattern in patterns:
        self.builder.add_definition(pattern)
    
    for region, pattern in zip(regions, patterns):
        patName = pattern.attributes['id']
        fill = f"url(#{patName})"
        
        # Create a style with this pattern as fill
        style = StyleCSS(f"overlay_{patName}", fill=fill, stroke="black",stroke_width=0.5)
        self.builder.add_style(style)
        
        # Get all paths for this region (handles non-contiguous)
        for path in region.trace_perimeter_cached(borders, f, style):
            retLayer += path.drawClosed()
    
    return retLayer


In [ ]:
#| export
@patch
def visualize_difference(terrain: Terrain,adjustment,thresholds = [-5, -1, 1, 5] ):
    """Visualize erosion with red (removal) and green (deposition)."""
    
   
    # 5 categories: heavy erosion, light erosion, minimal, light deposition, heavy deposition
      # Splits into 5 bins
    
    colors = [
        "#DC143C",  # Red - heavy erosion (< -5m)
        "#DC143C",  # Red - light erosion (-5 to -1m)
        "#808080",  # Gray - minimal change (-1 to 1m)
        "#32CD32",  # Green - light deposition (1 to 5m)
        "#32CD32",  # Green - heavy deposition (> 5m)
    ]
    
    # Bin adjustment values
    adj_indices = np.digitize(adjustment, thresholds)
    adj_indices = np.clip(adj_indices, 0, len(colors) - 1)
    
    # Create patterns with varying ball density
    patternGen = TerrainPatterns(self.terrain)
    patterns = patternGen.ballSpectrum(len(colors), fills=colors, prefix="erosion")
    
    # Generate overlay
    return self.makeOverlay(adj_indices, patterns)

this is my region code
```
# AUTOGENERATED! DO NOT EDIT! File to edit: ../../nbs/plots/02d_HexRegion.ipynb.

# %% auto 0
__all__ = ['BoundaryPoint', 'HexRegion', 'windy_edge', 'variable_windy_edge']

# %% ../../nbs/plots/02d_HexRegion.ipynb 2
import sys
import math
import numpy as np
import math
from collections import namedtuple
from dataclasses import dataclass, field
from fastcore.basics import patch



# %% ../../nbs/plots/02d_HexRegion.ipynb 3
from .primitives import MapCord, MapSize, MapRect, MapPath, PrimitiveDemo
from .cube import HexPosition
from .hex import Hex, HexGrid

from ..styles import StyleCSS,  SVGBuilder

# %% ../../nbs/plots/02d_HexRegion.ipynb 5
@dataclass
class HexRegion:
    """A set of adjacent hexes with computed perimeter boundaries."""
    hexes: set[int]  # Set of hex indices
    hex_grid: 'HexGrid'  # Reference to get hex objects

    def centroid_hex(self) -> int:
        """Return the hex index closest to the geometric center."""
        if not self.hexes:
            return None
        
        # Calculate centroid of all hex centers
        cx, cy = 0, 0
        for idx in self.hexes:
            h = self.hex_grid.hexes[idx]
            cx += h.center.x
            cy += h.center.y
        cx /= len(self.hexes)
        cy /= len(self.hexes)
        
        # Find hex closest to centroid
        closest = None
        min_dist = float('inf')
        for idx in self.hexes:
            h = self.hex_grid.hexes[idx]
            dist = (h.center.x - cx)**2 + (h.center.y - cy)**2
            if dist < min_dist:
                min_dist = dist
                closest = idx
        return closest
  
    @classmethod
    def fromPath(cls, grid: HexGrid, path: list[int]):
        """Create a HexRegion along a list of hex indices."""
        if not path:
            return cls(hexes=set(), hex_grid=grid)
        
        adds = set()
        
        # Process each segment of the path
        for i in range(len(path) - 1):
            start_idx = path[i]
            end_idx = path[i + 1]
            
            # Convert to HexPositions relative to start
            start_pos = grid.index_to_hexposition(start_idx, start_idx)  # (0,0,0)
            end_pos = grid.index_to_hexposition(end_idx, start_idx)
            
            # Get radial path from start to end
            hexes = start_pos.line_to(end_pos)
            
            # Convert back to indices and add
            for hexpos in hexes:
                idx = grid.hexposition_to_index(hexpos, start_idx)
                if idx >= 0:  # Valid index
                    adds.add(idx)
        
        # Don't forget the last hex
        adds.add(path[-1])
        
        return cls(hexes=adds, hex_grid=grid)


# %% ../../nbs/plots/02d_HexRegion.ipynb 6
@patch
def __or__(self: HexRegion, other: 'HexRegion') -> 'HexRegion':
    """Union: region1 | region2"""
    return HexRegion(self.hexes | other.hexes, self.hex_grid)

@patch
def __and__(self: HexRegion, other: 'HexRegion') -> 'HexRegion':
    """Intersection: region1 & region2"""
    return HexRegion(self.hexes & other.hexes, self.hex_grid)

@patch
def __sub__(self: HexRegion, other: 'HexRegion') -> 'HexRegion':
    """Difference: region1 - region2"""
    return HexRegion(self.hexes - other.hexes, self.hex_grid)

@patch
def __xor__(self: HexRegion, other: 'HexRegion') -> 'HexRegion':
    """Symmetric difference: region1 ^ region2"""
    return HexRegion(self.hexes ^ other.hexes, self.hex_grid)

@patch
def __contains__(self: HexRegion, idx: int) -> bool:
    """Membership: idx in region"""
    return idx in self.hexes

@patch
def __len__(self: HexRegion) -> int:
    """Size: len(region)"""
    return len(self.hexes)

@patch
def __iter__(self: HexRegion):
    """Iterate over hex indices"""
    return iter(self.hexes)


# %% ../../nbs/plots/02d_HexRegion.ipynb 7
@patch
def outside(self:HexRegion,ring=1):
    m = set()
    grid = self.hex_grid
    total = grid.nCols * grid.nRows
    for index in self.hexes:
        ring_hexpositions = HexPosition(0, 0, 0).ring(ring)
        neighbor_indices = [grid.hexposition_to_index(hp, index) for hp in ring_hexpositions]
        for neighbor in neighbor_indices:
            if neighbor >= 0 and neighbor < total and neighbor not in self.hexes:
                m.add(neighbor)
    return HexRegion(m,grid) # Filter out-of-bounds

@patch 
def apply(self:HexRegion,direction:HexPosition):
    m = set()
    grid = self.hex_grid
    total = grid.nCols * grid.nRows
    for index in self.hexes:
        #hp = HexPosition(0, 0, 0).ra(ring) + direction
        neighbor = grid.hexposition_to_index(direction, index) 
        if neighbor >= 0 and neighbor < total :
            m.add(neighbor)
    return HexRegion(m,grid)

@patch 
def shift(self:HexRegion,direction:HexPosition):
    m = set()
    grid = self.hex_grid
    total = grid.nCols * grid.nRows
    for index in self.hexes:
        #hp = HexPosition(0, 0, 0).ra(ring) + direction
        neighbor = grid.hexposition_to_index(direction, index) 
        if neighbor >= 0 and neighbor < total and neighbor not in self.hexes:
            m.add(neighbor)
    return HexRegion(m,grid)

# %% ../../nbs/plots/02d_HexRegion.ipynb 8
@patch
def inside(self:HexRegion,ring=1):
    out = self.outside().outside()
    m = self.hexes - out.hexes
    return HexRegion(m,self.hex_grid) # Filter out-of-bounds

# %% ../../nbs/plots/02d_HexRegion.ipynb 9
@patch
def styleHexes(self:HexRegion,style=StyleCSS):
    for h in self.hexes:
        self.hex_grid.hexes[h].style = style

# %% ../../nbs/plots/02d_HexRegion.ipynb 15
@patch
def styleRegions(self:HexGrid):
    ret = {}
    for i, h in enumerate(self.hexes):
        region = ret.get(h.style.name,HexRegion(set(),self))
        region.hexes.add(i)
        ret[h.style.name] = region
    return ret

# %% ../../nbs/plots/02d_HexRegion.ipynb 17
@patch
def contiguous(self: HexRegion) -> list[HexRegion]:
    """Split region into contiguous sub-regions."""
    grid = self.hex_grid
    remaining = set(self.hexes)
    regions = []
    
    while remaining:
        # Start a new region with any remaining hex
        start = next(iter(remaining))
        connected = set()
        frontier = {start}
        
        # Flood fill
        while frontier:
            current = frontier.pop()
            if current in connected:
                continue
            connected.add(current)
            remaining.discard(current)
            
            # Add neighbors that are in our region
            for hp in HexPosition(0, 0, 0).ring(1):
                neighbor_idx = grid.hexposition_to_index(hp, current)
                if neighbor_idx >= 0 and neighbor_idx in remaining:
                    frontier.add(neighbor_idx)
        
        regions.append(HexRegion(connected, grid))
    
    return regions


# %% ../../nbs/plots/02d_HexRegion.ipynb 19
BoundaryPoint = namedtuple('BoundaryPoint', ['hex_idx', 'vertex'])


# %% ../../nbs/plots/02d_HexRegion.ipynb 20
@patch
def trace_boundary(self: HexRegion, verbose=False) -> list[BoundaryPoint]:
    """Trace boundary with debug output."""
    if not self.hexes:
        return []
    
    grid = self.hex_grid
    
    # Find a boundary hex - PREFER edges facing off-grid
    start = None
    fallback = None
    
    for idx in self.hexes:
        for hp in HexPosition(0, 0, 0).ring(1):
            neighbor = grid.hexposition_to_index(hp, idx)
            dir_idx = hp.direction_index()
            
            if neighbor < 0:
                start = BoundaryPoint(idx, Hex._direction_to_vertices[dir_idx][0])
                if verbose:
                    print(f"Found off-grid start: {start}")
                break
            elif neighbor not in self.hexes and fallback is None:
                fallback = BoundaryPoint(idx, Hex._direction_to_vertices[dir_idx][0])
        if start is not None:
            break
    
    if start is None:
        start = fallback
        if verbose:
            print(f"Using fallback start: {start}")
    
    if start is None:
        print("No boundary found!")
        return []
    
    path = []
    current = start
    
    while True:
        path.append(current)
        if verbose and len(path) <= 20:
            print(f"Step {len(path)}: {current}")
        
        next_vertex = (current.vertex + 1) % 6
        
        # Find direction for this edge
        edge_dir = None
        for dir_idx, (v1, v2) in enumerate(Hex._direction_to_vertices):
            if v1 == current.vertex and v2 == next_vertex:
                edge_dir = dir_idx
                break
        
        if edge_dir is not None:
            hp = HexPosition.directions()[edge_dir]
            neighbor = grid.hexposition_to_index(hp, current.hex_idx)
            
            if verbose and len(path) <= 20:
                print(f"  Edge dir={edge_dir}, neighbor={neighbor}, in_region={neighbor in self.hexes if neighbor >= 0 else 'OOB'}")
            
            if neighbor >= 0 and neighbor in self.hexes:
                current = BoundaryPoint(neighbor, (next_vertex + 3) % 6)
                if verbose and len(path) <= 20:
                    print(f"  -> Jump to {current}")
            else:
                current = BoundaryPoint(current.hex_idx, next_vertex)
                if verbose and len(path) <= 20:
                    print(f"  -> Stay, advance to {current}")
        else:
            current = BoundaryPoint(current.hex_idx, next_vertex)
        
        if current == start:
            if verbose:
                print(f"Completed loop after {len(path)} steps")
            break
        
        if len(path) > len(self.hexes) * 6:
            print(f"Warning: path too long ({len(path)}), breaking")
            break
    
    return path


# %% ../../nbs/plots/02d_HexRegion.ipynb 21
@patch
def boundary_to_coords(self: HexRegion, path: list[BoundaryPoint]) -> list[MapCord]:
    """Convert (hex_index, vertex_index) path to MapCord list."""
    return [self.hex_grid.hexes[hex_idx].v[vertex_idx] for hex_idx, vertex_idx in path]

@patch 
def boundary_path(self: HexRegion, style=None) -> MapPath:
    """Get boundary as a MapPath ready for rendering."""
    if style is None:
        style = StyleCSS("boundary", fill="none", stroke="#333", stroke_width=2)
    
    path = self.trace_boundary()
    coords = self.boundary_to_coords(path)
    return MapPath(coords, style).closed()


# %% ../../nbs/plots/02d_HexRegion.ipynb 23
@patch
def trace_perimeter(self: HexRegion, debug=False, 
                   style=StyleCSS("perimeter_path", fill="none", 
                                 stroke="#ba3ca3ff", stroke_width=3)):
    """Trace perimeter using commonEdge to find boundary edges."""

    paths = []
       
    for subR in self.contiguous():
        path = subR.boundary_path(style=style)
        paths.append(path)
    
    return paths  # Return paths and empty gaps list


# %% ../../nbs/plots/02d_HexRegion.ipynb 24
@patch
def cloudLayer(self:HexGrid):
    retLayer = ""
    regions = self.styleRegions()
    for styleName, region in regions.items():
        style = self.builder.styles[styleName]
        for path in region.trace_perimeter(style=style):
            text = path.svg()
            retLayer += text
    return retLayer

# %% ../../nbs/plots/02d_HexRegion.ipynb 27
def _edge_key(p1: MapCord, p2: MapCord) -> tuple:
    """Canonical key for an edge - always ordered the same way."""
    if (p1.x, p1.y) < (p2.x, p2.y):
        return (round(p1.x, 2), round(p1.y, 2), round(p2.x, 2), round(p2.y, 2))
    return (round(p2.x, 2), round(p2.y, 2), round(p1.x, 2), round(p1.y, 2))

def _edge_is_forward(p1: MapCord, p2: MapCord) -> bool:
    """True if p1->p2 matches canonical order."""
    return (p1.x, p1.y) < (p2.x, p2.y)


# %% ../../nbs/plots/02d_HexRegion.ipynb 28
@patch
def trace_boundary_with_cache(self: HexRegion, borders: dict, f=None, style=None) -> MapPath:
    """Trace boundary using shared edge cache.
    
    Args:
        borders: Shared dict mapping edge_key -> list[MapCord] (the transformed points)
        f: Function (p1, p2) -> list[MapCord] to transform edge endpoints into path segment
        style: Style for the resulting MapPath
    """
    if style is None:
        style = StyleCSS("boundary", fill="none", stroke="#333", stroke_width=2)
    
    if f is None:
        f = lambda p1, p2: [p1, p2]  # Default: straight line
    
    boundary_points = self.trace_boundary()
    if not boundary_points:
        return MapPath([], style)
    
    all_coords = []
    
    for i in range(len(boundary_points)):
        bp1 = boundary_points[i]
        bp2 = boundary_points[(i + 1) % len(boundary_points)]
        
        p1 = self.hex_grid.hexes[bp1.hex_idx].v[bp1.vertex]
        p2 = self.hex_grid.hexes[bp2.hex_idx].v[bp2.vertex]
        
        key = _edge_key(p1, p2)
        forward = _edge_is_forward(p1, p2)
        
        if key not in borders:
            # Compute and cache in canonical direction
            if forward:
                borders[key] = f(p1, p2)
            else:
                borders[key] = f(p2, p1)
        
        # Get segment, reverse if needed
        segment = borders[key]
        if not forward:
            segment = list(reversed(segment))
        
        # Add points (skip first if not first segment to avoid duplicates)
        if i == 0:
            all_coords.extend(segment)
        else:
            all_coords.extend(segment[1:])
    
    return MapPath(all_coords, style).closed()


# %% ../../nbs/plots/02d_HexRegion.ipynb 29
@patch
def trace_perimeter_cached(self: HexRegion, borders: dict, f=None, 
                           style=StyleCSS("perimeter_path", fill="none", 
                                         stroke="#ba3ca3ff", stroke_width=3)):
    """Trace perimeter using shared border cache."""
    paths = []
    for subR in self.contiguous():
        path = subR.trace_boundary_with_cache(borders, f, style)
        paths.append(path)
    return paths


# %% ../../nbs/plots/02d_HexRegion.ipynb 30
@patch
def styleLayer(self: HexGrid, f=None):
    """Generate style layer with shared borders between regions."""
    retLayer = ""
    regions = self.styleRegions()
    borders = {}  # Shared cache across all regions
    
    for styleName, region in regions.items():
        style = self.builder.styles[styleName]
        for path in region.trace_perimeter_cached(borders, f, style):
            retLayer += path.drawClosed()
    
    return retLayer


# %% ../../nbs/plots/02d_HexRegion.ipynb 32
def windy_edge(iterations=2, offset_factor=0.15, seed=None):
    """Factory for windy edge transform function."""
    def transform(p1: MapCord, p2: MapCord) -> list[MapCord]:
        path = MapPath([p1, p2], None).make_windy(
            iterations=iterations, 
            offset_factor=offset_factor,
            seed=seed
        )
        return path.points
    return transform

def variable_windy_edge(iterations=2, offset_min=0.05, offset_max=0.2, seed=None):
    """Factory for variable windy edge transform."""
    def transform(p1: MapCord, p2: MapCord) -> list[MapCord]:
        path = MapPath([p1, p2], None).make_windy_variable(
            iterations=iterations,
            offset_min=offset_min,
            offset_max=offset_max,
            seed=seed
        )
        return path.points
    return transform

```

I want to have a make overlay for a list of regions similar to how we we make an overlay using def styleLayer(self: HexGrid, f=None):

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

I can help you create an overlay method for regions! To give you the best approach, let me understand what you need:

1. Should this new method take a list of `HexRegion` objects and apply patterns to all hexes within those regions?

2. Do you want each region to have its own pattern, or should all regions in the list share the same pattern?

3. Should it work similarly to `makeOverlay` where you pass in patterns, or more like `styleLayer` which uses a function?

Once you clarify these points, I can guide you on how to structure it!

1. each region should have its own pattern. both the regions and the patterns will be lists
2. because regions don't have to be continous one pattern per region.
3. i think we want to pass in a function with just the normal edges if neccessary this way we can make windy

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! So you want:
- A list of regions paired with a list of patterns (one pattern per region)
- Apply patterns to all hexes in each region
- Optionally pass in an edge transform function (like `windy_edge`)

Here's the signature I'm thinking:

```python
@patch
def overlayRegions(self: HexGrid, regions: list[HexRegion], 
                   patterns: list[SVGBuilder], f=None):
```

This would:
1. Loop through `zip(regions, patterns)`
2. For each region, get all hex indices in `region.hexes`
3. Apply the pattern to those hexes (similar to `makeOverlay`)
4. If `f` is provided, use it to transform boundaries (similar to `styleLayer`)

Does this match what you're looking for, or should it work differently?

you wrote


Ah, I understand now! You want to apply patterns to regions, where:

    Each region can have multiple disconnected paths (non-contiguous areas)
    All paths within the same region should use the same pattern
    Different regions get different patterns

So the signature would be:
Copied!

@patch
def overlayRegions(self: HexGrid, regions: list[HexRegion], 
                   patterns: list[SVGBuilder], f=None):
    """Apply patterns to regions. Each region gets one pattern applied to all its paths."""

And the implementation would:

    Loop through zip(regions, patterns)
    For each region, call region.trace_perimeter() to get all the paths
    Apply the same pattern to all paths from that region
    Optionally use f to transform the edges

Is that the right understanding?

Yes

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Here's the implementation:

```python
@patch
def overlayRegions(self: HexGrid, regions: list[HexRegion], 
                   patterns: list[SVGBuilder], f=None):
    """Apply patterns to regions. Each region gets one pattern applied to all its paths.
    
    Args:
        regions: List of HexRegion objects
        patterns: List of SVGBuilder patterns (one per region)
        f: Optional edge transform function (e.g., windy_edge())
    
    Returns:
        SVG string for the overlay layer
    """
    retLayer = ""
    borders = {}  # Shared cache across all regions
    
    # Add all pattern definitions
    for pattern in patterns:
        self.builder.add_definition(pattern)
    
    for region, pattern in zip(regions, patterns):
        patName = pattern.attributes['id']
        fill = f"url(#{patName})"
        
        # Create a style with this pattern as fill
        style = StyleCSS(f"overlay_{patName}", fill=fill, stroke="none")
        
        # Get all paths for this region (handles non-contiguous)
        for path in region.trace_perimeter_cached(borders, f, style):
            retLayer += path.drawClosed()
    
    return retLayer
```

This:
1. Uses the shared `borders` cache so adjacent regions share edge transforms
2. Adds all patterns to the SVG defs
3. For each region, traces all its paths (contiguous sub-regions) 
4. Applies the same pattern fill to all paths in that region
5. Uses `f` for edge transforms (pass `windy_edge()` for organic borders)

Usage example:
```python
# Create regions and patterns
regions = [region1, region2, region3]
patterns = terrainPatterns.ballDensity(levels=3, prefix="overlay")

# Straight edges
overlay_svg = grid.overlayRegions(regions, patterns)

# Windy edges
overlay_svg = grid.overlayRegions(regions, patterns, f=windy_edge(iterations=2))
```

great now we just need a functon that converts the nparray into index lists that we pass in for creating the hexes. I think numpy would have some sort of group index thing

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Yes! `np.where` is perfect for this. Here's a simple function:

```python
def indices_by_value(data: np.ndarray) -> dict[int, list[int]]:
    """Group array indices by their values.
    
    Args:
        data: Array of integer values (e.g., pattern indices)
    
    Returns:
        Dict mapping value -> list of indices where that value occurs
    """
    result = {}
    for val in np.unique(data):
        if val >= 0:  # Skip -1 or negative values (often used for "no pattern")
            result[int(val)] = np.where(data == val)[0].tolist()
    return result
```

Or to get regions directly:

```python
def regions_by_value(grid: HexGrid, data: np.ndarray) -> list[HexRegion]:
    """Convert data array into list of HexRegions, one per unique value.
    
    Returns regions in order of sorted unique values (0, 1, 2, ...).
    """
    regions = []
    for val in sorted(np.unique(data)):
        if val >= 0:
            indices = set(np.where(data == val)[0].tolist())
            regions.append(HexRegion(indices, grid))
    return regions
```

Usage:
```python
# Bin your data
pattern_indices = rainfall_selector_np(terrain.fields['rainfall'])

# Get regions (one per pattern level)
regions = regions_by_value(terrain.hexGrid, pattern_indices)

# Get matching patterns
patterns = terrainPatterns.ballDensity(levels=len(regions), prefix="rain")

# Apply overlay
overlay_svg = grid.overlayRegions(regions, patterns, f=windy_edge())
```

In [ ]:
#| export
# Create a terrain
@patch
def circusDemo(self:TerraDemo):
    sampleMap = TerraDemo().tiny()

    def rainfall_selector_np(values: np.ndarray) -> np.ndarray:
        # bin edges: <5 returns 0, 5-12 returns 1, 12-24 returns 2, etc.
        bins = [0.1, 5, 12, 24, 48]
        return np.digitize(values, bins) - 1 

    test_data = np.array([-1, 2, 8, 15, 30, 60, 0, 12, 48])

    indices = rainfall_selector_np(test_data)
    indices

    # Generate random rainfall data
    rainfall_data = np.random.uniform(-4, 60, len(sampleMap.elevations))
    sampleMap.fields['rainfall'] = rainfall_data

    # Get pattern indices
    pattern_indices = rainfall_selector_np(rainfall_data)
    regions = sampleMap.hexGrid.regions_by_value(pattern_indices)

    # Create patterns and overlay
    patternGen = TerrainPatterns(sampleMap)
    patterns = patternGen.ballScale(len(pattern_indices),fills=["#007fff","#d4ff00ff","#ee00ffff","#ff0099ff","#00ff1eff"])  # 5 levels
    patot = sampleMap.hexGrid.overlayRegions(regions, patterns)
    #print(patot)
    sampleMap.hexGrid.builder.adjust("regionfall",patot)
    #sampleMap.colorMap()
    #sampleMap.hexGrid.update()

    # View it
    #print(sampleMap.hexGrid.builder._header())
    return sampleMap.hexGrid.builder.show()


In [ ]:
TerraDemo().circusDemo()

In [ ]:
@patch
def circusDemo(self:TerraDemo):
    sampleMap = TerraDemo().tiny()

    def rainfall_selector_np(values: np.ndarray) -> np.ndarray:
        # bin edges: <5 returns 0, 5-12 returns 1, 12-24 returns 2, etc.
        bins = [0.1, 5, 12, 24, 48]
        return np.digitize(values, bins) - 1 

    test_data = np.array([-1, 2, 8, 15, 30, 60, 0, 12, 48])

    indices = rainfall_selector_np(test_data)
    indices

    # Generate random rainfall data
    rainfall_data = np.random.uniform(-4, 60, len(sampleMap.elevations))
    sampleMap.fields['rainfall'] = rainfall_data

    # Get pattern indices
    pattern_indices = rainfall_selector_np(rainfall_data)

    # Create patterns and overlay
    patternGen = TerrainPatterns(sampleMap)
    patterns = patternGen.ballScale(len(pattern_indices),fills=["#007fff","#d4ff00ff","#ee00ffff","#ff0099ff","#00ff1eff"])  # 5 levels
    patot = sampleMap.makeOverlay(pattern_indices, patterns)
    sampleMap.hexGrid.builder.adjust("rainfall",patot)
    #sampleMap.colorMap()
    #sampleMap.hexGrid.update()

    # View it
    #print(sampleMap.hexGrid.builder.xml())
    return sampleMap.hexGrid.builder.show()

In [ ]:
TerraDemo().circusDemo()

In [ ]:
#| export
@patch
def fillPattern(self:HexRegion,pattern:SVGBuilder,smooth=False):
    """Fill a region with a style.

     fill = f"url(#{patName})"

            ret = "<polygon points=\""
            hex = grid.hexes[i]
            for point in hex.vertices():
                ret += f"{point.x:.0f},{point.y:.0f} "
            ret += f"\" style=\"fill:{fill}\""
    
    
    """
    
    fill_layer = ""
    patName = pattern.attributes['id']
    fill = f"url(#{patName})"
    #fill = "#d4ff00eb"

    paths, gaps = self.trace_perimeter(style=StyleCSS("blank"),debug=False)
    #self.builder.add_style(pathstyle)

    pathLayer = ""
    for path in paths:
        if path.points[0].distance(path.points[-1]) < 40:
            cl = path.closed()
        else:
            cl = path
        
        pathLayer += "<path d=\""
        if smooth:
            curved = cl.make_windy(iterations=1, offset_factor=0.1, seed=42)
            smooth = curved.smooth(iterations=1)
            pathLayer += smooth.to_svg_path( close=True)
        else:
            
            pathLayer += cl.to_svg_path( close=True)
        pathLayer += f"\" style=\"fill:{fill}\""
        pathLayer += "/>"
        pathLayer += "\n"
    

    return pathLayer

    
    flowData = np.zeros(len(self.hexGrid.hexes)) - 1

    for i in self.hexes:
        #print(i)
        flowData[i] = 1

    flowData = [int(x) for x in flowData]
    # Create patterns and overlay
    patternGen = TerrainPatterns(self)
    patterns = patternGen.ballDensity(3,fills=fills)  # 5 levels
    self.makeOverlay(flowData, patterns)

In [ ]:
@patch
def fillPatternInverted(self:HexRegion, pattern:SVGBuilder,dest:SVGBuilder, smooth=False):
    """Fill OUTSIDE a region with a pattern."""
    
    # Get the region boundary
    paths, gaps = self.trace_perimeter(style=StyleCSS("blank"), debug=False)
    
    # Create mask definition
    mask_id = f"mask_{id(self)}"
    mask_def = f'<mask id="{mask_id}">'
    mask_def += '<rect x="0" y="0" width="10000" height="10000" fill="white"/>'
    
    # Add black paths for the region (these will be "cut out")
    for path in paths:
        cl = path.closed() if path.points[0].distance(path.points[-1]) < 40 else path
        mask_def += '<path d="'
        if smooth:
            curved = cl.make_windy(iterations=1, offset_factor=0.1, seed=42)
            smooth_path = curved.smooth(iterations=1)
            mask_def += smooth_path.to_svg_path(close=True)
        else:
            mask_def += cl.to_svg_path(close=True)
        mask_def += '" fill="black"/>'
    
    mask_def += '</mask>'
    
    # Add mask to definitions
    self.hexGrid.builder.add_definition_raw(mask_def)
    
    # Create filled rect with mask applied
    patName = pattern.attributes['id']
    fill = f"url(#{patName})"
    
    return f'<rect x="0" y="0" width="10000" height="10000" fill="{fill}" mask="url(#{mask_id})"/>'


In [ ]:
@patch 
def demoFill(self:TerraDemo): 
    """Practice building up coord."""

    sampleMap = self.sanFran()
    peaks = sampleMap.find_peaks(7,0)
    start = peaks[0]
    region = HexRegion(hexes=set([start]),  hex_grid=sampleMap.hexGrid)
    #region.add(start)
    #print(region.perimeter())
    fills=["#d4ff00eb","#ffb300ff","#ff0073ff","#9900ff97","#1e0e45eb"]
    patternGen = TerrainPatterns(sampleMap)
    patterns = patternGen.ballDensity(8,fills=fills)

    for level in range(3):

        levels = [x for x in range(len(sampleMap.elevations)-1) if sampleMap.elevationLevel(x) == (level-1)]

        region = HexRegion(hexes=set(levels), hex_grid=sampleMap.hexGrid)


        print(f"level {level} count {len(region.hexes)}")
        

        # Show the base terrain
        sampleMap.colorMap()
        sampleMap.hexGrid.update()
        #sampleMap.dot(region.perimeter())

        
          # 5 levels
        myPat = patterns[level+2]
        sampleMap.hexGrid.builder.add_definition(myPat)
        over = region.fillPattern(myPat,True)
        sampleMap.builder.adjust(f"level_{level}",over)
        
    sampleMap.builder.adjust("root","")
    #print(sampleMap.hexGrid.builder.xml())
    #return

    return sampleMap.hexGrid.builder.show()



In [ ]:
#| export
class SVGMask(Generatable):
    """SVG mask definition that can be added to SVGBuilder definitions"""
    
    def __init__(self, mask_id: str, width: int = 10000, height: int = 10000):
        self.mask_id = mask_id
        self.width = width
        self.height = height
        self.paths = []  # List of (path_d, fill_color) tuples
        self.background_color = "white"  # White = show, black = hide
    
    def add_path(self, path_d: str, fill: str = "black", smooth: bool = False):
        """Add a path to the mask. Black areas will be hidden."""
        self.paths.append((path_d, fill))
        return self
    
    def set_background(self, color: str):
        """Set mask background color (white=show, black=hide)"""
        self.background_color = color
        return self
    
    def generate(self) -> str:
        """Generate the mask definition XML"""
        ret = f'<mask id="{self.mask_id}">\n'
        ret += f'  <rect x="0" y="0" width="{self.width}" height="{self.height}" fill="{self.background_color}"/>\n'
        
        for path_d, fill in self.paths:
            ret += f'  <path d="{path_d}" fill="{fill}"/>\n'
        
        ret += '</mask>'
        return ret


In [ ]:
#| export
@patch
def fillPatternInverted(self: HexRegion, pattern: SVGBuilder, smooth: bool = False):
    """Fill OUTSIDE a region with a pattern using a mask."""
    
    # Get the region boundary
    paths = self.trace_perimeter(style=StyleCSS("blank"), debug=False)
    
    # Create mask
    mask_id = f"mask_{id(self)}"
    mask = SVGMask(mask_id)
    
    # Add paths to mask (black = hide these areas)
    for path in paths:
        cl = path.closed() if path.points[0].distance(path.points[-1]) < 40 else path
        
        if smooth:
            curved = cl.make_windy(iterations=1, offset_factor=0.1, seed=42)
            smooth_path = curved.smooth(iterations=1)
            path_d = smooth_path.to_svg_path(close=True)
        else:
            path_d = cl.to_svg_path(close=True)
        
        mask.add_path(path_d, fill="black")
    
    # Add mask to builder definitions
    self.hexGrid.builder.add_definition(mask)
    
    # Create filled rect with mask applied
    patName = pattern.attributes['id']
    fill = f"url(#{patName})"
    
    return f'<rect x="0" y="0" width="10000" height="10000" fill="{fill}" mask="url(#{mask_id})"/>'


In [ ]:
#| export
@patch
def hatchLines(self: TerrainPatterns, angle: float = 45, spacing: float = 8, 
               stroke_width: float = 1.5, color: str = "#3d9fc0ff") -> SVGBuilder:
    """Create diagonal line hatch pattern."""
    
    # Pattern tile needs to be large enough for one full line repeat
    size = spacing * 2
    
    # Calculate line endpoints based on angle
    # For 45°, lines go from bottom-left to top-right
    body = f'''
    <g>
        <line x1="0" y1="{size}" x2="{size}" y2="0" 
              stroke="{color}" stroke-width="{stroke_width}"/>
        <line x1="-{size}" y1="{size}" x2="{size}" y2="-{size}" 
              stroke="{color}" stroke-width="{stroke_width}"/>
    </g>
    '''
    
    aBuilder = SVGBuilder()
    aBuilder.blockTag = "pattern"
    aBuilder.width = size
    aBuilder.height = size
    aBuilder.attributes = {
        'id': 'hatch_lines',
        'patternUnits': 'userSpaceOnUse',
        'patternTransform': f'rotate({angle})'
    }
    aBuilder.updateLayers([body])
    
    return aBuilder


### adding elements

In [ ]:
#| export
@patch
def island(self:TerraDemo):
    mySize = MapSize(480,480)
    myBounds = MapRect(MapCord(0,0), mySize)
    sampleMap =  Terrain(myBounds,radius=15,path = "volcano.svg")
    
    sampleMap.elevations += sampleMap.volcano(center=267,adjusted=500,num_rings=6,variability=0.5,initial_threshold=0.4)

    levels = [x for x in range(len(sampleMap.elevations)-1) if sampleMap.elevationLevel(x) >= 0]

    region = HexRegion(hexes=set(levels), hexGrid=sampleMap.hexGrid)
    fills=["#3300ffeb","#3d9fc0ff"]
    patternGen = TerrainPatterns(sampleMap)
    patterns = patternGen.ballDensity(4,fills=fills)

    sampleMap.colorMap()
    sampleMap.hexGrid.update()

    myPat = patterns[1]
    #myPat = patternGen.hatchLines(angle=25, spacing=8, stroke_width=1.5, color="#3d9fc0ff")

    sampleMap.hexGrid.builder.add_definition(myPat)
    over = region.fillPatternInverted(myPat,True)
    #sampleMap.builder.adjust(f"root","")
    sampleMap.builder.adjust(f"ocean",over)
    return sampleMap

@patch
def demoAquatic(self:TerraDemo):
    sampleMap = self.island()

    

    #print(sampleMap.hexGrid.builder.xml())
    

    return sampleMap.hexGrid.builder.show()

In [ ]:
TerraDemo().demoAquatic()

In [ ]:
SeaPatterns = SVGPatternLoader(patterns_dir="data/patterns/nautical/")
SeaPatterns.get_available_patterns()

In [ ]:
#| export
@patch
def mapElement(sampleMap:Terrain,bounds:MapCord,name="compass_1",prefix="merright",style=StyleCSS("base", 
                        fill="#27ae60",  # Green
                        stroke="#c12121ff")): 
    
    # Show the terrain first
    sampleMap.colorMap()
    sampleMap.hexGrid.update()

    dim = bounds.dimensons.width
    xoffset = bounds.origin.x
    yoffset = bounds.origin.y

    
    #
    #name = 'compass_1.svg' #'merright_1svg'.
    aPat = SeaPatterns.load_pattern(name + '.svg', f"{name}_aid", prefix=prefix)

    # Override to green - all paths will have class="merright_path"
    green_style = StyleCSS(f"{prefix}_path", 
                        fill=style.properties["fill"],  # Green
                        stroke=style.properties["stroke"])
    aPat.attributes['patternUnits'] = 'userSpaceOnUse'
    aPat.attributes['patternContentUnits'] = 'objectBoundingBox'
    aPat.attributes['patternTransform'] = f'translate({xoffset},{yoffset}) scale({dim/aPat.width})'
    aPat.add_style(green_style)
    #aPat.add_style(green_style, prefix="merright")
    
    sampleMap.hexGrid.builder.add_definition(aPat)
    over = f'<rect x="{xoffset}" y="{yoffset}" width="{dim}" height="{dim}" fill="#0080ff1f"   opacity="0.8"/>'
    over = f'<rect x="{xoffset}" y="{yoffset}" width="{dim}" height="{dim}" fill="url(#{name}_aid)"   opacity="0.8"/>'
    return over
    

    

    return sampleMap.hexGrid.builder.show()

In [ ]:
@patch
def demoAquaticDetails(self:TerraDemo): 
    mySize = MapSize(480,480)
    myBounds = MapRect(MapCord(0,0), mySize)
    sampleMap = Terrain(myBounds, radius=15, path="volcano.svg")
    
    sampleMap.elevations += sampleMap.volcano(center=267, adjusted=500, num_rings=6, variability=0.5, initial_threshold=0.4)

    # Show the terrain first
    sampleMap.colorMap()
    sampleMap.hexGrid.update()

    dim = 150
    xoffset = 600
    yoffset = 500

    
    # Load and add the compass pattern with smaller scale
    myColors = StyleCSS("base", fill="#ae2778ff",   stroke="#c12121ff")
    
    over = sampleMap.mapElement(MapRect(MapCord(xoffset,yoffset),MapSize(dim,dim)),name="merright_1",prefix="merright",style=myColors)
    
    # Add compass as overlay instead of replacing everything
    #sampleMap.hexGrid.builder.adjust("root","")
   
    sampleMap.hexGrid.builder.adjust("compass2", over)

    myColors = StyleCSS("base", fill="#8cae27ff",   stroke="#c12121ff")
    
    over = sampleMap.mapElement(MapRect(MapCord(20,yoffset),MapSize(160,220)),name="ten_4",prefix="ten_4",style=myColors)
    sampleMap.hexGrid.builder.adjust("ten", over)


    file_name = "tmp/tp_AquaticDetail_demo.svg"
    with open(file_name, 'w') as file_object:
        file_object.write(sampleMap.hexGrid.builder.xml())
    #print(sampleMap.hexGrid.builder.xml())
    

    return sampleMap.hexGrid.builder.show()

In [ ]:
TerraDemo().demoAquaticDetails()

In [ ]:
name = 'merright_1.svg'
aPat = SeaPatterns.load_pattern(name, "compass_1")
print(StyleCSS.generate(SeaPatterns.find_css(name)))

In [ ]:
name = 'merright_1.svg'
aPat = SeaPatterns.load_pattern(name, "compass_1")
print(StyleCSS.generate(SeaPatterns.find_css(name)))

right. but the region generates a series of paths of contious regions. each of these regions should have the same pattern

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Ah, I understand now! You want to apply patterns to regions, where:

1. Each region can have **multiple disconnected paths** (non-contiguous areas)
2. All paths within the **same region** should use the **same pattern**
3. Different regions get different patterns

So the signature would be:

```python
@patch
def overlayRegions(self: HexGrid, regions: list[HexRegion], 
                   patterns: list[SVGBuilder], f=None):
    """Apply patterns to regions. Each region gets one pattern applied to all its paths."""
```

And the implementation would:
1. Loop through `zip(regions, patterns)` 
2. For each region, call `region.trace_perimeter()` to get all the paths
3. Apply the **same pattern** to all paths from that region
4. Optionally use `f` to transform the edges

Is that the right understanding?

## working with terrain

In [ ]:
from HexMagic.terrain import Terrain

In [ ]:
from HexMagic.primitives import MapSize, MapCord, MapRect

In [ ]:
@patch
def styleLayerOrdered(self: HexGrid, styles: list[StyleCSS], f=None, smooth=False):
    """Generate style layer with styles applied in specified order.
    
    Args:
        styles: List of StyleCSS objects in the order to render them
        f: Optional edge transformation function
    """
    retLayer = ""
    regions = self.styleRegions()
    borders = {}
    rendered = set()

    # First: render specified styles in order
    for style in styles:
        styleName = style.name
        if styleName not in regions:
            continue
        rendered.add(styleName)
        region = regions[styleName]
        for path in region.trace_perimeter_cached(borders, f, style):
            if smooth:
                retLayer += path.sbg()
            else:
                retLayer += path.drawClosed()

    # Then: render any remaining regions
    for styleName, region in regions.items():
        if styleName in rendered:
            continue
        style = self.builder.styles.get(styleName)
        for path in region.trace_perimeter_cached(borders, f, style):
            retLayer += path.drawClosed()

    return retLayer


In [ ]:
def demoTerr():

    mySize = MapSize(480,480)
    myBounds = MapRect(MapCord(0,0), mySize)
    sampleMap, plates =  Terrain.fromSeeds(myBounds,radius=15)

    sampleMap.colorMap()
    sgrid = sampleMap.hexGrid
    sgrid.builder.adjust("regions", sgrid.styleLayerOrdered(
        styles=sampleMap.colorLevels,
        f=unique_windy_edge(iterations=3)))
    #sampleMap.hexGrid.update()

    return sampleMap.hexGrid.builder.show() 

In [ ]:
??HexGrid.styleLayer

we need a version of styleLayer where we pass in a list of how the styles are to be applied. for instance for our elevations we want to go in order of elevation

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Got it! You want to control the **rendering order** of styled regions. Here's a version that takes an ordered list:

```python
@patch
def styleLayerOrdered(self: HexGrid, style_order: list[str], f=None):
    """Generate style layer with styles applied in specified order.
    
    Args:
        style_order: List of style names in the order to render them
        f: Optional edge transformation function
    """
    retLayer = ""
    regions = self.styleRegions()
    borders = {}  # Shared cache across all regions

    for styleName in style_order:
        if styleName not in regions:
            continue
        region = regions[styleName]
        style = self.builder.styles[styleName]
        for path in region.trace_perimeter_cached(borders, f, style):
            retLayer += path.drawClosed()

    return retLayer
```

Usage would be:

```python
# Define elevation order (lowest to highest, so higher renders on top)
elevation_order = ["sealevel", "Lowland", "Plains", "Hills", "Foothills", 
                   "Highlands", "Mountains", "Peaks", "Summits", "Snow"]

sgrid.builder.adjust("regions", 
    sgrid.styleLayerOrdered(elevation_order, f=windy_edge(iterations=2, offset_factor=0.1)))
```

Or if you want it even more flexible, you could pass tuples of `(style_name, pattern)` to also control which pattern each level gets:

```python
@patch
def styleLayerWithPatterns(self: HexGrid, style_configs: list[tuple[str, SVGBuilder|None]], f=None):
    """Apply styles in order, optionally with pattern fills."""
    # ...
```

Which approach fits your use case better?

In [ ]:
demoTerr()

In [ ]:
demoTerr()

I think we want to pass in something like the colorlevels or something generated from 

I think we want to pass in something like the list of styles we get from seaborn

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Right! So you'd pass the list of `StyleCSS` objects directly, and extract the names from them:

```python
@patch
def styleLayerOrdered(self: HexGrid, styles: list[StyleCSS], f=None):
    """Generate style layer with styles applied in specified order.
    
    Args:
        styles: List of StyleCSS objects in the order to render them
        f: Optional edge transformation function
    """
    retLayer = ""
    regions = self.styleRegions()
    borders = {}

    for style in styles:
        styleName = style.name
        if styleName not in regions:
            continue
        region = regions[styleName]
        for path in region.trace_perimeter_cached(borders, f, style):
            retLayer += path.drawClosed()

    return retLayer
```

Usage:

```python
# Generate elevation palette
elevation_styles = StyleCSS.seaborn("YlOrRd", levels=10)

# Apply in order (seaborn already returns them in order)
sgrid.builder.adjust("regions", 
    sgrid.styleLayerOrdered(elevation_styles, f=windy_edge(iterations=2, offset_factor=0.1)))
```

This way the colors/styles and their order come from the same source.

---

**Note:** Some earlier messages in our conversation were truncated due to context limits (originally 55 messages, now 13).

We also probbly want to draw the regions which didn't have styles passed in

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good point! We should render the specified styles in order, then any remaining regions after:

```python
@patch
def styleLayerOrdered(self: HexGrid, styles: list[StyleCSS], f=None):
    """Generate style layer with styles applied in specified order.
    
    Args:
        styles: List of StyleCSS objects in the order to render them
        f: Optional edge transformation function
    """
    retLayer = ""
    regions = self.styleRegions()
    borders = {}
    rendered = set()

    # First: render specified styles in order
    for style in styles:
        styleName = style.name
        if styleName not in regions:
            continue
        rendered.add(styleName)
        region = regions[styleName]
        for path in region.trace_perimeter_cached(borders, f, style):
            retLayer += path.drawClosed()

    # Then: render any remaining regions
    for styleName, region in regions.items():
        if styleName in rendered:
            continue
        style = self.builder.styles.get(styleName)
        for path in region.trace_perimeter_cached(borders, f, style):
            retLayer += path.drawClosed()

    return retLayer
```

This way:
1. Ordered styles render first (in specified order)
2. Any regions with styles not in the list render afterwards (using their registered style from `builder.styles`)

---

**Note:** Some earlier messages were removed due to context limits (originally 56 messages, now 14).

so I need a different random seed for each


##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

You could pass a seed derived from each style's name:

```python
@patch
def styleLayerOrdered(self: HexGrid, styles: list[StyleCSS], f=None):
    """Generate style layer with styles applied in specified order."""
    retLayer = ""
    regions = self.styleRegions()
    borders = {}
    rendered = set()

    for style in styles:
        styleName = style.name
        if styleName not in regions:
            continue
        rendered.add(styleName)
        region = regions[styleName]
        # Create edge function with seed based on style name
        edge_f = f(seed=hash(styleName)) if f else None
        for path in region.trace_perimeter_cached(borders, edge_f, style):
            retLayer += path.drawClosed()

    for styleName, region in regions.items():
        if styleName in rendered:
            continue
        style = self.builder.styles.get(styleName)
        edge_f = f(seed=hash(styleName)) if f else None
        for path in region.trace_perimeter_cached(borders, edge_f, style):
            retLayer += path.drawClosed()

    return retLayer
```

Then make `windy_edge` return a factory when given a seed:

```python
def windy_edge(iterations=3, offset_factor=0.15, seed=None):
    rng = random.Random(seed)
    def transform(p1, p2):
        # use rng instead of random module
        ...
    return transform
```

Or pass a callable factory:

```python
sgrid.styleLayerOrdered(elevation_styles, 
    f=lambda seed: windy_edge(iterations=2, offset_factor=0.1, seed=seed))
```